In [1]:
#Here I will show what it might look like all set up to push data from BQ as means to automate YouTube updates
#Specifically Tagging
from YouTube import YouTube
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
%load_ext google.cloud.bigquery

In [2]:
#initialize youtube oauth object
y = YouTube(oauth = True)

Loading Credentials From File...
Saving Credentials for Future Use...
done


In [3]:
pandas_gbq.context.project = 'bigquery-public-data'
pandas_gbq.context.dialect = 'standard'

In [3]:
project_id = 'natural-caster-352615' #not sure if this will be umg-us or something else
client = bigquery.Client(project = project_id) 
query = '''
    SELECT name, SUM(number) as count
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    GROUP BY name
    ORDER BY count DESC
    LIMIT 5
''' #this is a public data set, replace it with a query of youtube urls (to get video ids)
client.query(query).result().to_dataframe() #optional to_dataframe() method

,name,count
0,James,5054074
1,John,4910976
2,Robert,4763102
3,Michael,4396482
4,William,3939912


In [3]:
#I am going to manually pull in IDs here for the example
df = pd.read_csv('YouTube Top Videos.csv')
df['video_id'] = df.video_url.apply(lambda x: str(x).split('=')[1])
#get a small sample to avoid query limits
df = df.head(10)

In [4]:
df

,video_title,views_td,video_url,latest_report_date,video_id
0,Stay Up,656.0,https://www.youtube.com/watch?v=xJzmHmm9mn4,2022-07-15,xJzmHmm9mn4
1,Sama Ajana,656.0,https://www.youtube.com/watch?v=yY20TnYv9Ck,2022-07-15,yY20TnYv9Ck
2,Cast Your Fate To The Wind,656.0,https://www.youtube.com/watch?v=ooMMSQkWF3Q,2022-07-15,ooMMSQkWF3Q
3,Guilty Hypocrites,656.0,https://www.youtube.com/watch?v=_0xxH4JDIA0,2022-07-15,_0xxH4JDIA0
4,Volver A Empezar (Remastered 2016),656.0,https://www.youtube.com/watch?v=KIgeZrGE7Ic,2022-07-15,KIgeZrGE7Ic
5,Cada Vez Que Nos Miramos,656.0,https://www.youtube.com/watch?v=Awu2256KMFg,2022-07-15,Awu2256KMFg
6,Hoje Você Me Disse,656.0,https://www.youtube.com/watch?v=5Wteqgh822k,2022-07-15,5Wteqgh822k
7,Cler Achel,656.0,https://www.youtube.com/watch?v=-9bIoRXMIGU,2022-07-15,-9bIoRXMIGU
8,Can't Get You Off My Mind,656.0,https://www.youtube.com/watch?v=32-TAIlXFyI,2022-07-15,32-TAIlXFyI
9,[I Know] I'm Losing You,656.0,https://www.youtube.com/watch?v=OzW8hx62Bpc,2022-07-15,OzW8hx62Bpc


In [5]:
#pull all video tags in df example
df['video_tags'] = df['video_id'].apply(lambda x: y.video_meta(ids = str(x), key = 'tags')) #uses youtube class video meta method
df

,video_title,views_td,video_url,latest_report_date,video_id,video_tags
0,Stay Up,656.0,https://www.youtube.com/watch?v=xJzmHmm9mn4,2022-07-15,xJzmHmm9mn4,"[Hit-Boy, ヒットボウイ, Sage The Gemini, ＳａｇｅＴｈｅＧｅｍｉ..."
1,Sama Ajana,656.0,https://www.youtube.com/watch?v=yY20TnYv9Ck,2022-07-15,yY20TnYv9Ck,"[Disiz, ディシズラペスト, ディシズ・ラ・ペスト, Flag, Itineraire..."
2,Cast Your Fate To The Wind,656.0,https://www.youtube.com/watch?v=ooMMSQkWF3Q,2022-07-15,ooMMSQkWF3Q,"[David Benoit, The Best Smooth Jazz Ever, Cast..."
3,Guilty Hypocrites,656.0,https://www.youtube.com/watch?v=_0xxH4JDIA0,2022-07-15,_0xxH4JDIA0,"[Said The Whale, Little Mountain, Guilty Hypoc..."
4,Volver A Empezar (Remastered 2016),656.0,https://www.youtube.com/watch?v=KIgeZrGE7Ic,2022-07-15,KIgeZrGE7Ic,"[Nino Bravo, Augusto Alguero, Mi Tierra, Volve..."
5,Cada Vez Que Nos Miramos,656.0,https://www.youtube.com/watch?v=Awu2256KMFg,2022-07-15,Awu2256KMFg,"[Camarón De La Isla, カマロンデライスラ, カマロン・デ・ラ・イスラ, ..."
6,Hoje Você Me Disse,656.0,https://www.youtube.com/watch?v=5Wteqgh822k,2022-07-15,5Wteqgh822k,"[Maria, Clara, Menina, Bonita, Hoje, Você, Me,..."
7,Cler Achel,656.0,https://www.youtube.com/watch?v=-9bIoRXMIGU,2022-07-15,-9bIoRXMIGU,"[Tinariwen, Aman Iman, Cler Achel]"
8,Can't Get You Off My Mind,656.0,https://www.youtube.com/watch?v=32-TAIlXFyI,2022-07-15,32-TAIlXFyI,"[Mary J. Blige, Mary / Share My World / The To..."
9,[I Know] I'm Losing You,656.0,https://www.youtube.com/watch?v=OzW8hx62Bpc,2022-07-15,OzW8hx62Bpc,"[The Temptations, Motown Chartbusters Vol 1, (..."


In [7]:
#count number of tags for each video
df['num_tags'] = df['video_tags'].apply(lambda x: len(x))
#filter on videos without informational tags, usually just <8
low_tag = df[df['num_tags']<8]
low_tag.reset_index()

,index,video_title,views_td,video_url,latest_report_date,video_id,video_tags,num_tags
0,0,Stay Up,656.0,https://www.youtube.com/watch?v=xJzmHmm9mn4,2022-07-15,xJzmHmm9mn4,"[Hit-Boy, ヒットボウイ, Sage The Gemini, ＳａｇｅＴｈｅＧｅｍｉ...",7
1,1,Sama Ajana,656.0,https://www.youtube.com/watch?v=yY20TnYv9Ck,2022-07-15,yY20TnYv9Ck,"[Disiz, ディシズラペスト, ディシズ・ラ・ペスト, Flag, Itineraire...",6
2,2,Cast Your Fate To The Wind,656.0,https://www.youtube.com/watch?v=ooMMSQkWF3Q,2022-07-15,ooMMSQkWF3Q,"[David Benoit, The Best Smooth Jazz Ever, Cast...",3
3,3,Guilty Hypocrites,656.0,https://www.youtube.com/watch?v=_0xxH4JDIA0,2022-07-15,_0xxH4JDIA0,"[Said The Whale, Little Mountain, Guilty Hypoc...",3
4,4,Volver A Empezar (Remastered 2016),656.0,https://www.youtube.com/watch?v=KIgeZrGE7Ic,2022-07-15,KIgeZrGE7Ic,"[Nino Bravo, Augusto Alguero, Mi Tierra, Volve...",4
5,7,Cler Achel,656.0,https://www.youtube.com/watch?v=-9bIoRXMIGU,2022-07-15,-9bIoRXMIGU,"[Tinariwen, Aman Iman, Cler Achel]",3
6,8,Can't Get You Off My Mind,656.0,https://www.youtube.com/watch?v=32-TAIlXFyI,2022-07-15,32-TAIlXFyI,"[Mary J. Blige, Mary / Share My World / The To...",3
7,9,[I Know] I'm Losing You,656.0,https://www.youtube.com/watch?v=OzW8hx62Bpc,2022-07-15,OzW8hx62Bpc,"[The Temptations, Motown Chartbusters Vol 1, (...",3


In [8]:
#some method to update tags on low videos goes here
#carlos wants the updates to consist of related search terms but 
#I think there would need to be a request to youtube for that data
#Other ideas involve using twitter api to find similar key words and append those to current list
new_tags = ['new tags here']
#once you have your tag lists updated, reupload:
for i,j in enumerate(low_tag['video_id']):
    current = low_tag['video_tags'].iloc[i]
    current = current + new_tags
    #update the tags. 'cms_usage' should always be true when working with content made by an artist
    y.update(id = j, key = 'tags', content = current, cms_usage = True)

HttpError: <HttpError 404 when requesting https://youtube.googleapis.com/youtube/v3/videos?part=snippet&onBehalfOfContentOwner=UCuxOToTM4F-DKaUpoAd38wg&alt=json returned "The specified content owner account was not found.". Details: "[{'message': 'The specified content owner account was not found.', 'domain': 'youtube.parameter', 'reason': 'contentOwnerAccountNotFound', 'location': 'onBehalfOfContentOwner', 'locationType': 'parameter'}]">

In [ ]:
#as expected, without cms I get an owner not found error/access error
#should be resolved with proper login